The enum keyword is used to declare an enumeration, a distinct type that consists of a set of named constants called the enumerator list. 

In [1]:
from enum import Enum

Month = Enum('Month', ('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'))

In [6]:
for name, member in Month.__members__.items():
    print(name, '=>', member, ',', member.value)

Jan => Month.Jan , 1
Feb => Month.Feb , 2
Mar => Month.Mar , 3
Apr => Month.Apr , 4
May => Month.May , 5
Jun => Month.Jun , 6
Jul => Month.Jul , 7
Aug => Month.Aug , 8
Sep => Month.Sep , 9
Oct => Month.Oct , 10
Nov => Month.Nov , 11
Dec => Month.Dec , 12


In [5]:
Month.__members__.items()

odict_items([('Jan', <Month.Jan: 1>), ('Feb', <Month.Feb: 2>), ('Mar', <Month.Mar: 3>), ('Apr', <Month.Apr: 4>), ('May', <Month.May: 5>), ('Jun', <Month.Jun: 6>), ('Jul', <Month.Jul: 7>), ('Aug', <Month.Aug: 8>), ('Sep', <Month.Sep: 9>), ('Oct', <Month.Oct: 10>), ('Nov', <Month.Nov: 11>), ('Dec', <Month.Dec: 12>)])

In [7]:
class Gender(Enum):
    Male = 0
    Female = 1

class Student(object):
    def __init__(self, name, Gender):
        self.name = name
        self.gender = Gender

In [10]:
Gender.Female

<Gender.Female: 1>

In [20]:
alice = Student("Alice", Gender.Female)

In [21]:
alice.name

'Alice'

In [22]:
alice.gender

<Gender.Female: 1>

The usage of Python @property.

In [63]:
class User(object):
    
    @property
    def password(self):
        # Use to get the attribute
        self._password = 'm'.join(self._password.split("s"))
        return self._password
    
    @password.setter
    def password(self, value):
        # Use to set the attribute
        if len(value) <= 8:
            raise ValueError('Password length too short.')
        self._password = value

In [64]:
user = User() # new instance of user object

In [65]:
user.password = "badpass"

ValueError: Password length too short.

In [66]:
user.password = "goodpassword"

In [67]:
user.password

'goodpammword'

In [68]:
abs(-11)

11

In [80]:
def fn(self, name):
    return name

In [81]:
User = type('User', (object,), dict(get_name=fn))

In [82]:
user = User()
user.get_name('my name')

'my name'

`Select` has result from the db, but `Insert, Update, Delete` will return the interation of row.

In [89]:
import sqlite3
conn = sqlite3.connect('file:/tmp/data.db', uri=True)

In [90]:
cur = conn.cursor()

In [98]:
cur = cur.execute('SELECT * FROM todo_list;')

In [99]:
cur.rowcount

-1

In [100]:
cur.fetchall()

[(1, 'hello', 0)]

In [96]:
cur.rowcount

-1

Define what is the User object should be like.

In [118]:
class User(Model):
    __table__ = 'users'
    
    id = IntegerField(primary_key=True)
    name = StringField()
    # id, name, __table__ is will be the attribute of the class not for the class instance.
    # The difference of that is will can call the the attribute direactly user User.id. no more need instance.

NameError: name 'Model' is not defined

In [115]:
# create instance
user = User(id=123, name='Alice')
# save to the DB
user.insert()
# query the user object
users = User.findAll()

In [219]:
# Define the basic class of ORM Model
class Model(dict):
    def __init__(self, **kwargs):
        super(Model, self).__init__(**kwargs)
        
    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError("'Model' object not has attribute {}".format(key))
    
    def __setattr__(self, key, value):

        self[key] = value

In [220]:
# Model class inherited dict, the instance of Model is the dict.
user_dict = Model(id=123, name='Alice')

In [221]:
user_dict.age
user_dict

AttributeError: 'Model' object not has attribute age

In [209]:
def fn(name):
    print
User = type('User', (object,), dict(func = lambda self, n: print(n)))

In [211]:
user = User()
user.func("hello")
user.a

hello


AttributeError: 'User' object has no attribute 'a'

In [44]:
class Field(object):
    def __init__(self, column_type, primary_key, default):
        self.column_type = column_type
        self.primary_key = primary_key
        self.default = default
    def __repr__(self):
        return '<{}, {}>'.format(self.__class__.__name__, self.column_type)

In [45]:
Field('Text', 'NOT TEXT', True)

<Field, Text>

In [90]:
class TextField(Field):
    
    def __init__(self, column_type='TEXT NOT NULL', default="", primary_key=False):
        super(TextField, self).__init__(column_type, primary_key, default)
        
class IntegerField(Field):
    
    def __init__(self, column_type='INTEGER NOT NULL', default=1, primary_key=False):
        super(IntegerField, self).__init__(column_type, primary_key, default)
        
class BooleanField(Field):
    def __init__(self, column_type='BOOLEAN NOT NULL', default=False):
        super(BooleanField, self).__init__(column_type, False, default)

In [91]:
TextField()
IntegerField()
BooleanField()

<BooleanField, BOOLEAN NOT NULL>

In [102]:
import inspect
class ModelMetaclass(type):
    def __new__(cls, name, bases, attrs):
        if name == 'Model':
            return type.__new__(cls, name, bases, attrs)
        table_name = name
        fields = []
        mappings = dict()
        for k, v in attrs.items():
            if isinstance(v, Field):
                mappings[k] = v
                if v.primary_key:
                    primary_key = k
                else:
                    fields.append(k)
                    
        for k in mappings.keys():
            attrs.pop(k)
        escaped_fields = list(map(lambda f: '{}'.format(f), fields))
        attrs['PRIMARY_KEY'] = primary_key
        attrs['MAPPINGS'] = mappings
        attrs['TABLE_NAME'] = table_name
        attrs['FIELD'] = fields
        return type.__new__(cls, name, bases, attrs)        
# Define the basic class of ORM Model
class Model(dict, metaclass = ModelMetaclass):
    def __init__(self, **kwargs):
        super(Model, self).__init__(**kwargs)
        
    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError("'Model' object not has attribute {}".format(key))
    
    def __setattr__(self, key, value):
        self[key] = value
    
    def _get_value(self, key):
        value = getattr(self, key, None)
        if value is None:
            # If instance have not set the value of the key
            # try to get the default value from Field object.
            field = self.MAPPINGS[key]
            if field.default is not None:
                value = field.default
                setattr(self, key, value)
        return value
    
    def create_table(self):
        values = []
        for key, field in self.MAPPINGS.items():
            sql = ' '.join([key, field.column_type, 'PRIMARY KEY' if field.primary_key else ''])
            values.append(sql)
        #cur.execute(sql, args)
        print("execute")
        return 'CREATE TABLE IF NOT EXISTS {} ({})'.format(self.TABLE_NAME, ','.join(values))
    
    def save(self):
        args = list(map(self._get_value, self.MAPPINGS))
        columns = list(map(lambda k: k, self.MAPPINGS))
        #cur.execute(sql, args)
        print(args)
        return 'INSERT INTO {} ({}) VALUES({})'.format(self.TABLE_NAME, ', '.join(columns), ','.join('?'*len(columns)))
    
    @classmethod
    def find(self, primary_key):
        pass
        
      
class Todo(Model):
    id = IntegerField(primary_key=True)
    context = TextField()
    flag = BooleanField()
    def __init__(self):
        self.create_table()

In [103]:
todo = Todo()

execute


'?,?,?'